In [2]:
from torch.nn import Module, Sequential, Linear, LeakyReLU, Sigmoid, BCELoss
from torch.utils.data import DataLoader, TensorDataset
from torch import device, cuda
from torch.optim import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_recall_curve, confusion_matrix
import pandas as pd
from tqdm import tqdm
import torch
from time import time
import os

import sys
sys.path.append('..')  
from src.utils.RawDataReader import RawDataReader
from src.utils.SingleHeadModel import SingleHeadModel



In [2]:
reader = RawDataReader()
labels = reader.get_labels()
datasets, means, stds = reader.get_normalised_dataset()

In [3]:
r = 5 

f1_scores = {}
f1_score_validations = {}
models = {}
num_labels = labels.shape[1]

validation_split = 0.05
test_split = 0.20
batch_size = 2048
learning_rate = 0.001
epochs = 99
hidden_layers = 10

num_folds = 10

random_state = 1

device = device("cuda" if cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [4]:
curr_dataset = datasets[r]
mean = means[r]
std = stds[r]
del datasets
del means
del stds

In [1]:
%%writefile ../src/utils/SingleHeadModel.py
from torch.nn import Module, Sequential, Linear, LeakyReLU, Sigmoid

class SingleHeadModel(Module):
    def __init__(self, feature_count:int, hidden_count:int=5):
        super(SingleHeadModel, self).__init__()

        self.slope = 0.001

        self.model = Sequential(
            *[self.linear_block(feature_count,feature_count) for _ in range(hidden_count)],
            Linear(in_features=feature_count, out_features=1),
            Sigmoid()
        )

    def linear_block(self, in_features, out_features):
        return Sequential(
            Linear(in_features=in_features, out_features=out_features),
            LeakyReLU(negative_slope=self.slope)
        )
    
    def forward(self, x):
        return self.model(x)


Writing ../src/utils/SingleHeadModel.py


In [6]:
stratified_folds = StratifiedKFold(n_splits=num_folds, random_state=random_state, shuffle=True)

In [9]:
# curr_time = time()
# os.mkdir(f"../models/{curr_time}")

feature_count = curr_dataset.shape[1]

# for label_index in range(num_labels):
#     curr_label = labels[:,[label_index]]
#     x_train, x_test, y_train, y_test = train_test_split(curr_dataset, curr_label, test_size=test_split, random_state=random_state, stratify=curr_label)
#     x_train = x_train.to(device)
#     y_train = y_train.to(device)
#     x_test = x_test.to(device)
#     y_test = y_test.to(device)


#     train_dataset = TensorDataset(x_train, y_train)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#     test_dataset = TensorDataset(x_test, y_test)

#     model = SingleHeadModel(
#         feature_count=feature_count,
#         hidden_count=hidden_layers
#     ).to(device)

#     criterion = BCELoss()
#     optimizer = Adam(model.parameters(), lr=learning_rate)

#     for epoch in tqdm(range(epochs), ascii=False, desc="Epoch"):
#         model.train()
#         total_loss = 0
#         for batch_features, batch_labels in train_loader:
#             optimizer.zero_grad()
#             predictions = model(batch_features)
#             loss = criterion(predictions, batch_labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss
#             total_loss = total_loss / len(train_loader)

#         model.eval()
#         predictions = model(x_test)
                
#         f1 = f1_score(y_test.cpu().detach(), (predictions.cpu().detach()>0.5).float(),average='micro')
#         f1_scores[(r,label_index,epoch)] = f1
            
#     print(f"R:{r}, label:{label_index}, --> f1:{f1}")

#     # models[(r,label_index,fold_index)] = model
#     torch.save(model.state_dict(), f'../models/{curr_time}/SingleHeadModel_norm_{hidden_layers}hidden_r{r}_label{label_index}.pth')
#     model.to('cpu')
#     x_train.to('cpu')
#     y_train.to('cpu')
#     x_test.to('cpu')
#     y_test.to('cpu')

#     del model
#     del optimizer
#     del criterion
#     del train_dataset
#     del train_loader
#     del test_dataset
#     del x_train
#     del y_train
#     del x_test
#     del y_test

In [14]:
curr_time = '1701454903.1140416'
prediction_array = []
for label in range(19):
    model = SingleHeadModel(
        feature_count=feature_count,
        hidden_count=hidden_layers
    ).to(device)
    model.load_state_dict(torch.load(f'../models/{curr_time}/SingleHeadModel_norm_{hidden_layers}hidden_r{r}_label{label}.pth'))
    
    model.eval()
    predictions = (model(curr_dataset.to(device)) > 0.5).float()
    prediction_array.append(predictions)
    
    # predictions.sum()
    # # labels[:,[label]].sum()lab


NameError: name 'confusion_matrix' is not defined

In [ ]:
cat_output = torch.cat(prediction_array, dim=1).to('cpu')
f1_score(labels,cat_output,average="micro")

0.9415369258709846